# Приложение для прогноза напряжения (Stress)

In [2]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt

In [3]:
# Загружаем отфильтрованный датафрейм (3 колонки)
dfc = pd.read_csv(r"C:\Users\piken\df_clean.csv")

In [4]:
print(dfc.head(3))

     C_wt        dL    Stress
0  77.342  0.000006  7.121136
1  77.342  0.000012  7.133818
2  77.342  0.000112  7.529003


In [5]:
# Загружаем нормализованный датафрейм
df = pd.read_csv(r"dfnorm.csv")

In [6]:
print(df.head(3))

       C_wt        dL    Stress
0 -1.026045 -1.814900 -2.247307
1 -1.026045 -1.814860 -2.247055
2 -1.026045 -1.814207 -2.239318


## Входы и выходы для моделей

In [7]:
from sklearn.preprocessing import PowerTransformer
from joblib import dump, load

powertransformer_y = PowerTransformer()  
powertransformer_x = PowerTransformer()
y_label = ['C_wt', 'dL']
powertransformer_y.fit(dfc[y_label])
powertransformer_x.fit(dfc.drop(y_label, axis = 1))

# Сохраним масштабаторы для x и y
# from joblib import dump, load
dump(powertransformer_y, 'powertransformer_y.joblin')
dump(powertransformer_x, 'powertransformer_x.joblin')

# scaler_y.inverse_transform(y) - применим позже в приложении

['powertransformer_x.joblin']

In [8]:
# проверим x
powertransformer_y_l = load('powertransformer_y.joblin')
x1 = powertransformer_y_l.inverse_transform(np.array([0.1,0.99]).reshape(1,-1))
x1[0][0], x1[0][1]

(79.77677349884928, 0.617722666931493)

In [9]:
# проверим y
powertransformer_x_l = load('powertransformer_x.joblin')
y1 = powertransformer_x_l.inverse_transform(np.array([0.5]).reshape(1,-1))
y1[0][0]

892.501062193717

## Создадим тренировочную и тестовую выборки

In [10]:
from sklearn.model_selection import train_test_split

# Скопируем датафрейм в x. y будет равен колонке Stress
X = df.copy()
y = X.pop('Stress')
y_label = ['C_wt', 'dL']
y.shape, X.shape

((42485,), (42485, 2))

In [11]:
# Разделим датасет на тренировочную и тестовую выборку
# Активируем перемешивание образцов, train - 70%, test - 30%

X_train, X_test, y_train, y_test = train_test_split(X,y,
                                                    test_size=0.3, 
                                                    random_state=17, 
                                                   )

## Алгоритм регрессии K БЛИЖАЙШИХ СОСЕДЕЙ

In [12]:
from sklearn.model_selection import train_test_split
from sklearn.model_selection import GridSearchCV
from sklearn.neighbors import KNeighborsRegressor
from sklearn.model_selection import cross_val_score, GridSearchCV, KFold
from sklearn import metrics
import time # время выполнения кода

In [13]:
# К Ближайших соседей - KNeighborsRegressor
model = KNeighborsRegressor()
grid = {'n_neighbors' : range(1, 50, 1), 
          'weights' : ['uniform', 'distance'],
          'algorithm' : ['auto']
          }
gsc = GridSearchCV(model, grid, n_jobs=-1, cv=5,verbose = 1)

In [14]:
# Время выполнения
start = time.time() # точка отсчета времени
gsc.fit(X_train,  y_train)
model = gsc.best_estimator_
model.fit(X_train,  y_train)
y_pred = model.predict(X_test)
model_name = 'KNeighborsRegressor'

# Вывод времени выполнения кода программы
end = time.time() - start
print(end)

Fitting 5 folds for each of 98 candidates, totalling 490 fits
32.32035565376282


In [15]:
# Параметры модели
model

KNeighborsRegressor(n_neighbors=3, weights='distance')

In [16]:
# Сохраним модель в формате joblib
from joblib import dump, load
dump(KNeighborsRegressor, 'KNN_m.joblib')

['KNN_m.joblib']

In [17]:
# Сохраним модель в формате pickle
model_KNNM = load('pickle_model_KNN.pkl')
model_KNNM

KNeighborsRegressor(n_neighbors=3, weights='distance')

## Приложение

In [18]:
def input_variable():
  x1 = float(input('Введите концентрацию углерода (C_wt), %: '))
  x2 = float(input('Введите относительное удлинение (dL), %: '))
  return x1,x2

def input_proc(X):   
    print('Вызов модели KNN.pkl')
    res = model_KNNM.predict(X) #модель должна принять значения из двух столбцов
    return res 

In [20]:
def app_model():
  scaler_x = load('powertransformer_x.joblin')
  scaler_y = load('powertransformer_y.joblin') 
  model_l = load('pickle_model_KNN.pkl') 
  print('Данное приложение прогнозирует значение напряжения (Stress)')
  for i in range(100):
    try:
      print('Для прогноза введите 1, для выхода - 2')
      check = input()
      
      if check == '1':
        print('Введите данные:')
        X = input_variable()
        X = scaler_y.transform(np.array(X).reshape(1,-1))
        print(['Прогноз значения напряжения (Stress), МПа'])
        print(scaler_x.inverse_transform(input_proc(X).reshape(1,-1)))

      elif check == '2':
        break
      else:
        print('Повторите выбор')
    except:
      print('Неверные данные. Повторите операцию')
app_model()

Данное приложение прогнозирует значение напряжения (Stress)
Для прогноза введите 1, для выхода - 2
1
Введите данные:
Введите концентрацию углерода (C_wt), %: 81
Введите относительное удлинение (dL), %: 0.6
['Прогноз значения напряжения (Stress), МПа']
Вызов модели KNN.pkl
[[1053.72349874]]
Для прогноза введите 1, для выхода - 2
2
